# Create DTA Instance

Creates a new DTA entity in the DTA table.

**IMPORTANT:** A DTA is library-type agnostic - it's a container for ALL library types
(transfer_variables, test_concepts, codelists, etc.). The downstream notebooks
(save_draft, approve_dta) iterate over ALL configured library types.

**Sources:**
- `HISTORICAL` - From historical tsDTA processing (workflow_state=APPROVED)
- `UI` - From UI-initiated DTA creation (workflow_state=NOT_STARTED)

**What it does:**
1. Reads distinct trial metadata from primary silver table
2. Auto-generates DTA ID as UUID
3. Checks for MANUAL_REVIEW_REQUIRED records
4. Sets DTA status = MANUAL_REVIEW or ACTIVE accordingly
5. Creates DTA entity with audit columns
6. Creates DTA_WORKFLOW record for historical (auto-approved)
7. Creates DTA_APPROVAL_TASK records:
   - HISTORICAL: Single SYSTEM_APPROVED task (auto-approved) - represents automatic DTA Major creation
   - UI: JNJ_DAE + VENDOR tasks (PENDING) - manual approval required
8. Sets task values for downstream processing (source, dta_ids, library_type="ALL")


In [ ]:
# Cell 1: Imports
import json
import uuid
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, BooleanType
from clinical_data_standards_framework.utils import save_with_audit

print("="*80)
print("CREATE DTA INSTANCE")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
silver_schema = globals_dict['silver_schema']
gold_schema = globals_dict['gold_schema']

# Versioning configuration
versioning_enabled = versioning_dict.get('enabled', False)
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')
library_tables = versioning_dict.get('library_tables', [])

# Validate versioning config - must have library tables configured
if not library_tables:
    raise ValueError(
        "No library_tables configured in versioning config. "
        "Check that md_config_cache is populated correctly with job_populate_config_cache."
    )

print(f"Catalog: {catalog}")
print(f"Silver Schema: {silver_schema}")
print(f"Gold Schema: {gold_schema}")
print(f"Versioning Enabled: {versioning_enabled}")
print(f"Library Types: {[lib.get('library_type') for lib in library_tables]}")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Parameters from Job (passed via base_parameters)
# NOTE: DTA creation is library-type agnostic - one DTA contains ALL library types
# The downstream notebooks (save_draft, approve_dta) iterate over ALL library types

try:
    source = dbutils.widgets.get("source") or "HISTORICAL"
    trial_id_param = dbutils.widgets.get("trial_id")
    stream_type_param = dbutils.widgets.get("data_stream_type")
    provider_param = dbutils.widgets.get("data_provider_name")
except Exception as e:
    raise ValueError(
        f"Could not get job parameters: {e}. "
        "This notebook must be run as part of job_cdm_dta_create with proper parameters."
    )

print(f"\nSource: {source}")
print(f"Trial ID filter: {trial_id_param or '(all from data)'}")
print(f"Stream Type filter: {stream_type_param or '(all from data)'}")
print(f"Provider filter: {provider_param or '(all from data)'}")


In [ ]:
# Cell 4: Define Table Schemas

def get_dta_schema():
    """
    =============================================================================
    SCHEMA: dta (Gold Layer - Data Transfer Agreement)
    =============================================================================
    This table stores the core metadata for each Trial-Specific Data Transfer 
    Agreement (tsDTA). Each record represents a unique data transfer contract 
    between J&J and a data provider for a specific trial and data stream.
    
    The table includes identifiers for the trial and data stream, references to 
    the parent document, the provider, the transfer type, and the current status 
    of the DTA (e.g., Draft, Review, Approved).
    
    Version metadata and audit timestamps support governance and traceability.
    This table is the authoritative record of all active and historical DTAs 
    used to drive ingestion rules and metadata validation.
    
    Version Pointers:
        - version: Active version (draft or major) - what UI displays
        - current_draft_version: Latest draft in Silver (e.g., "1.0-DTA001-draft3")
        - base_template_version: Library version branched from (e.g., "1.0")
    
    Note: latest_major_version was removed - now derived from md_version_registry
    via: SELECT version FROM md_version_registry WHERE dta_id = :id AND version_type = 'DTA_APPROVED' AND status = 'ACTIVE'
    
    Status Alignment (1:1 with md_version_registry.status):
        - DRAFT: Being edited (effective_end_ts = NULL)
        - ACTIVE: Approved and in use (effective_end_ts = NULL)
        - PROMOTED: Elevated to template (effective_end_ts = NULL)
        - ARCHIVED: No longer active (effective_end_ts = timestamp)
    
    DTA Name Generation:
        - Historical import: {trial_id}_{data_stream_type}_{data_provider_name}
        - UI clone: <Change_this>_{trial_id}_{data_stream_type}_{data_provider_name}
    =============================================================================
    """
    return StructType([
        # Primary & Business Keys
        StructField("dta_id", StringType(), False),  # UUID primary key
        StructField("dta_number", StringType(), False),  # Sequential ID (DTA001, DTA002) for display
        StructField("dta_name", StringType(), True),  # User-friendly name for UI
        # Lineage
        StructField("parent_document_id", StringType(), True),  # FK to source tsDTA document
        # Business Key: trial_id + data_stream_type + data_provider_name
        StructField("trial_id", StringType(), True),  # Clinical trial ID (e.g., VAC18193RSV3001)
        StructField("data_stream_type", StringType(), True),  # Data category (Labs, Adverse Events)
        StructField("data_provider_name", StringType(), True),  # Vendor (LabCorp, Covance, PPD)
        # Status & Workflow (status aligns 1:1 with md_version_registry.status)
        StructField("status", StringType(), False),  # DRAFT, ACTIVE, PROMOTED, ARCHIVED
        StructField("workflow_state", StringType(), False),  # NOT_STARTED, IN_REVIEW, APPROVED, REJECTED
        StructField("workflow_iteration", IntegerType(), True),  # Current approval cycle (1, 2, 3...)
        # Version Pointers (latest_major_version removed - derived from md_version_registry)
        StructField("version", StringType(), True),  # Active version for UI display
        StructField("current_draft_version", StringType(), True),  # Latest draft in Silver
        StructField("base_template_version", StringType(), True),  # Branched from library version
        StructField("notes", StringType(), True),  # General notes about the DTA
        # Audit columns
        StructField("created_by_principal", StringType(), True),  # User who created
        StructField("created_ts", TimestampType(), True),  # Creation timestamp
        StructField("last_updated_by_principal", StringType(), True),  # User who last modified
        StructField("last_updated_ts", TimestampType(), True),  # Last modification timestamp
        StructField("databricks_job_id", StringType(), True),  # Job ID for lineage
        StructField("databricks_job_name", StringType(), True),  # Job name for lineage
        StructField("databricks_run_id", StringType(), True)  # Run ID for lineage
    ])

def get_dta_workflow_schema():
    """
    =============================================================================
    SCHEMA: dta_workflow (Gold Layer - Workflow Tracking)
    =============================================================================
    This table tracks the approval lifecycle for DTAs across all stakeholders 
    involved in the metadata governance process.
    
    It models workflow steps between J&J reviewers, external data providers 
    (vendors), and J&J librarians, who are responsible for promoting finalized 
    metadata into the official library versions.
    
    The table captures workflow identifiers, statuses, and audit timestamps for 
    initiation and updates. It provides the structured sequence of approval 
    events required to move a DTA from Draft to Review to Approved to Published 
    Metadata Library.
    =============================================================================
    """
    return StructType([
        StructField("dta_workflow_id", StringType(), False),  # UUID primary key
        StructField("dta_id", StringType(), False),  # FK to dta table
        StructField("workflow_iteration", IntegerType(), False),  # Cycle number (1, 2, 3...)
        StructField("workflow_status", StringType(), False),  # NOT_STARTED, IN_REVIEW, APPROVED, REJECTED
        StructField("summary_comment", StringType(), True),  # Workflow notes/rejection reasons
        StructField("initiated_ts", TimestampType(), True),  # When workflow cycle started
        StructField("closed_ts", TimestampType(), True),  # When workflow cycle completed
        # Audit columns
        StructField("created_by_principal", StringType(), True),
        StructField("created_ts", TimestampType(), True),
        StructField("last_updated_by_principal", StringType(), True),
        StructField("last_updated_ts", TimestampType(), True),
        StructField("databricks_job_id", StringType(), True),
        StructField("databricks_job_name", StringType(), True),
        StructField("databricks_run_id", StringType(), True)
    ])

def get_dta_approval_task_schema():
    """
    =============================================================================
    SCHEMA: dta_approval_task (Gold Layer - Approval Tasks)
    =============================================================================
    Individual approval tasks within a DTA workflow.
    
    HISTORICAL imports:
        - Single SYSTEM_APPROVED task (is_auto_approve=True, status=APPROVED)
        - Represents automatic DTA Major creation for pre-approved historical data
        - No manual approval needed since data is already validated
    
    UI flows:
        - JNJ_DAE task (manual approval, approval_order=1)
        - VENDOR task (manual approval, approval_order=2)
        - When BOTH approve, DTA Major is created by the approval API
        - No SYSTEM_APPROVED task for UI - promotion is triggered by last approver
    
    NOTE: Audit columns are added automatically by save_with_audit()
    =============================================================================
    """
    return StructType([
        StructField("approval_task_id", StringType(), False),      # UUID PK
        StructField("dta_workflow_id", StringType(), False),       # FK to dta_workflow
        StructField("dta_id", StringType(), False),                # FK to dta (denormalized)
        StructField("approver_role", StringType(), False),         # JNJ_DAE, VENDOR, SYSTEM_APPROVED
        StructField("assigned_to_principal", StringType(), True),  # User email (NULL for SYSTEM)
        StructField("approval_status", StringType(), False),       # PENDING, APPROVED, REJECTED
        StructField("approval_order", IntegerType(), False),       # Sequence (1, 2, 99)
        StructField("is_auto_approve", BooleanType(), True),       # True for SYSTEM_APPROVED
        StructField("approval_comment", StringType(), True),       # Comment from approver
        StructField("approved_ts", TimestampType(), True)          # When approved/rejected
        # NOTE: Audit columns added automatically by save_with_audit()
    ])

print("Schemas defined")


In [ ]:
# Cell 5: Configure Table Names and Find Existing Silver Tables
# DTA entity is library-type agnostic - it's a container for ALL library types
# We find ALL existing silver tables and will UNION them to get DTA contexts

# DTA tables (Gold layer)
dta_table = f"{catalog}.{gold_schema}.dta"
dta_workflow_table = f"{catalog}.{gold_schema}.dta_workflow"
dta_approval_task_table = f"{catalog}.{gold_schema}.dta_approval_task"

# Reference tables for normalized values (token overlap matching)
vendor_table = f"{catalog}.{gold_schema}.md_vendor"
data_stream_table = f"{catalog}.{gold_schema}.md_data_stream"

# Registry table from config
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

# Find ALL existing silver tables with data
existing_silver_tables = []
print("Searching for existing silver tables with data...")
for lib in library_tables:
    table_name = lib.get('silver_table')
    table_schema = lib.get('silver_schema', silver_schema)
    full_table = f"{catalog}.{table_schema}.{table_name}"
    
    if spark.catalog.tableExists(full_table):
        count = spark.table(full_table).count()
        if count > 0:
            existing_silver_tables.append({
                'table': full_table,
                'library_type': lib.get('library_type'),
                'count': count
            })
            print(f"  ✓ Found {lib.get('library_type')}: {full_table} ({count} rows)")
        else:
            print(f"  ⏭ Skipping {lib.get('library_type')}: table exists but empty")
    else:
        print(f"  ⏭ Skipping {lib.get('library_type')}: table does not exist")

if not existing_silver_tables:
    raise ValueError(
        f"No silver tables with data found. Available library tables: "
        f"{[lib.get('silver_table') for lib in library_tables]}. "
        f"Run at least one processor (tsDTA, OA, etc.) before creating DTA."
    )

print(f"\n✓ Found {len(existing_silver_tables)} silver table(s) with data")
print(f"DTA Table: {dta_table}")
print(f"DTA Workflow Table: {dta_workflow_table}")
print(f"DTA Approval Task Table: {dta_approval_task_table}")
print(f"Registry Table: {registry_table}")
print(f"\nAll Library Tables (from config):")
for lib in library_tables:
    lib_table = f"{catalog}.{lib.get('schema', gold_schema)}.{lib.get('name')}"
    print(f"  {lib.get('library_type')}: {lib_table}")


In [ ]:
# Cell 6: Read Silver Data and Get Distinct DTAs
# UNION all existing silver tables to get DTA contexts
# One parent_document_id = One DTA (parent_document_id is the source file, e.g., ZIP file)
# Uses NORMALIZED vendor/data_stream names from reference tables (token overlap matching)
print(f"\n{'='*80}")
print("Reading silver data for DTA metadata...")
print(f"{'='*80}")

# Union all existing silver tables to get DTA contexts
df_union = None
for silver_info in existing_silver_tables:
    df_silver = spark.table(silver_info['table'])
    
    # Select common context columns - parent_document_id is ALWAYS used
    df_context = df_silver.select(
        "parent_document_id",
        "trial_id", 
        "data_stream_type", 
        "data_provider_name"
    )
    
    if df_union is None:
        df_union = df_context
    else:
        df_union = df_union.unionByName(df_context)
    
    print(f"  Added contexts from {silver_info['library_type']}")

# ============================================================================
# NORMALIZATION: Join with reference tables to get canonical vendor/data_stream names
# This ensures that variant names (e.g., "Labcorp CLS" and "Labcorp") map to the same DTA
# ============================================================================
print("\n  Applying vendor/data_stream normalization...")

# Load reference tables if they exist
df_vendors = None
df_streams = None

if spark.catalog.tableExists(vendor_table):
    df_vendors = spark.table(vendor_table).select(
        F.col("vendor_name").alias("canonical_vendor"),
        F.col("aliases").alias("vendor_aliases")
    )
    print(f"    ✓ Loaded vendor reference table: {df_vendors.count()} vendors")

if spark.catalog.tableExists(data_stream_table):
    df_streams = spark.table(data_stream_table).select(
        F.col("data_stream_name").alias("canonical_stream"),
        F.col("aliases").alias("stream_aliases")
    )
    print(f"    ✓ Loaded data stream reference table: {df_streams.count()} streams")

# Apply normalization using array_contains for alias matching
if df_vendors is not None:
    # Explode aliases and join to find canonical vendor name
    df_union = df_union.join(
        df_vendors,
        F.array_contains(F.col("vendor_aliases"), F.col("data_provider_name")) |
        (F.col("canonical_vendor") == F.col("data_provider_name")),
        "left"
    ).select(
        "parent_document_id",
        "trial_id",
        "data_stream_type",
        F.coalesce(F.col("canonical_vendor"), F.col("data_provider_name")).alias("data_provider_name")
    )
    print(f"    ✓ Applied vendor normalization")

if df_streams is not None:
    # Join to find canonical stream name
    df_union = df_union.join(
        df_streams,
        F.array_contains(F.col("stream_aliases"), F.col("data_stream_type")) |
        (F.col("canonical_stream") == F.col("data_stream_type")),
        "left"
    ).select(
        "parent_document_id",
        "trial_id",
        F.coalesce(F.col("canonical_stream"), F.col("data_stream_type")).alias("data_stream_type"),
        "data_provider_name"
    )
    print(f"    ✓ Applied data stream normalization")

# ============================================================================
# GROUP BY parent_document_id to ensure ONE DTA per source file
# Take the first non-null values for trial_id, data_stream_type, data_provider_name
# This prevents duplicate DTAs when tsDTA and OA have slightly different values
# ============================================================================
print("\n  Grouping by parent_document_id (one DTA per source file)...")

df_dta_contexts = df_union.groupBy("parent_document_id").agg(
    F.first("trial_id", ignorenulls=True).alias("trial_id"),
    F.first("data_stream_type", ignorenulls=True).alias("data_stream_type"),
    F.first("data_provider_name", ignorenulls=True).alias("data_provider_name")
)

print(f"  ✓ Grouped to {df_dta_contexts.count()} unique parent_document_id(s)")

# If parameters provided, filter to specific context
if trial_id_param:
    df_dta_contexts = df_dta_contexts.filter(F.col("trial_id") == trial_id_param)
if stream_type_param:
    df_dta_contexts = df_dta_contexts.filter(F.col("data_stream_type") == stream_type_param)
if provider_param:
    df_dta_contexts = df_dta_contexts.filter(F.col("data_provider_name") == provider_param)

dta_contexts = df_dta_contexts.collect()
print(f"Found {len(dta_contexts)} DTA context(s) to process")


In [ ]:
# Cell 7: Ensure Tables Exist
now = datetime.now()

# Create DTA table if not exists
if not spark.catalog.tableExists(dta_table):
    print(f"Creating DTA table: {dta_table}")
    empty_df = spark.createDataFrame([], schema=get_dta_schema())
    empty_df.write.format("delta").mode("overwrite").saveAsTable(dta_table)
    print(f"Created {dta_table}")
else:
    print(f"DTA table exists: {dta_table}")

# Create DTA_WORKFLOW table if not exists
if not spark.catalog.tableExists(dta_workflow_table):
    print(f"Creating DTA_WORKFLOW table: {dta_workflow_table}")
    empty_df = spark.createDataFrame([], schema=get_dta_workflow_schema())
    empty_df.write.format("delta").mode("overwrite").saveAsTable(dta_workflow_table)
    print(f"Created {dta_workflow_table}")
else:
    print(f"DTA_WORKFLOW table exists: {dta_workflow_table}")


In [ ]:
# Cell 8: Create DTA Instances
print(f"\n{'='*80}")
print("Creating DTA instances...")
print(f"{'='*80}")

# Get the current max DTA number from existing table
def get_next_dta_number(spark, dta_table, offset=0):
    """Get the next sequential DTA number (DTA001, DTA002, etc.)"""
    try:
        # Check if table exists first
        if not spark.catalog.tableExists(dta_table):
            print(f"  ⚠️ DTA table does not exist yet, starting from DTA001")
            return f"DTA{(1 + offset):03d}"
        
        # Query for max number
        result = spark.sql(f"""
            SELECT MAX(CAST(REGEXP_EXTRACT(dta_number, 'DTA([0-9]+)', 1) AS INT)) as max_num
            FROM {dta_table}
        """).first()
        
        max_num = result.max_num if result and result.max_num is not None else 0
        next_num = max_num + 1 + offset
        
        print(f"  📊 Current max DTA number: {max_num}, Next: DTA{next_num:03d}")
        return f"DTA{next_num:03d}"
        
    except Exception as e:
        print(f"  ❌ ERROR getting DTA number: {e}")
        print(f"  ⚠️ Defaulting to DTA{(1 + offset):03d}")
        return f"DTA{(1 + offset):03d}"

dta_records = []
workflow_records = []
approval_task_records = []  # Approval tasks for workflow
created_dta_ids = []
created_dta_numbers = []  # Track DTA numbers for version tag

for idx, ctx in enumerate(dta_contexts):
    parent_document_id = ctx["parent_document_id"]  # Always present (source file ID)
    trial_id = ctx["trial_id"]
    stream_type = ctx["data_stream_type"]
    provider = ctx["data_provider_name"]
    
    print(f"\nProcessing: {trial_id} / {stream_type} / {provider} (Doc: {parent_document_id[:8]}...)")
    
    # Build filter condition - always filter by parent_document_id (one DTA per source file)
    filter_condition = (F.col("parent_document_id") == parent_document_id)
    
    # Check for MANUAL_REVIEW_REQUIRED records across all silver tables
    review_count = 0
    total_count = 0
    for silver_info in existing_silver_tables:
        df_silver = spark.table(silver_info['table'])
        # Check if status column exists in this table
        if "status" in df_silver.columns:
            review_count += df_silver.filter(
                filter_condition &
                (F.col("status") == "MANUAL_REVIEW_REQUIRED")
            ).count()
        total_count += df_silver.filter(filter_condition).count()
    
    needs_review = review_count > 0
    print(f"  Total records: {total_count}, Needs review: {review_count}")
    
    # Generate UUID for DTA
    dta_id = str(uuid.uuid4())
    
    # Generate sequential DTA number (DTA001, DTA002, etc.)
    dta_number = get_next_dta_number(spark, dta_table, offset=idx)
    
    created_dta_ids.append(dta_id)
    created_dta_numbers.append(dta_number)
    
    # Determine status, workflow_state, and version pointers based on source
    # Base library version - what this DTA branches from
    base_template_version = "1.0"  # TODO: Could query latest DTA_TEMPLATE from registry
    
    if source == "HISTORICAL":
        # Historical: Already approved, will go directly to DTA_APPROVED
        status = "MANUAL_REVIEW" if needs_review else "ACTIVE"
        workflow_state = "APPROVED"
        notes = f"Created from historical tsDTA data. Records needing review: {review_count}/{total_count}"
        # Version pointers for HISTORICAL (will be set after DTA_APPROVED creation)
        current_draft_version = None  # No draft phase for historical
        version = None  # Will be set by create_historical_dta_major
    else:  # UI
        # UI: Starts as draft in Silver
        status = "DRAFT"
        workflow_state = "NOT_STARTED"
        notes = "Created from UI"
        # Version pointers for UI (draft starts in Silver)
        draft_version = f"{base_template_version}-{dta_number}-draft1"
        current_draft_version = draft_version
        version = draft_version  # Points to Silver draft
    
    # Create DTA record
    dta_record = {
        "dta_id": dta_id,
        "dta_number": dta_number,  # Sequential DTA number for version tags
        "parent_document_id": parent_document_id,  # Link to source document (for historical lineage)
        "trial_id": trial_id,
        "data_stream_type": stream_type,
        "data_provider_name": provider,
        "status": status,
        "workflow_state": workflow_state,
        "workflow_iteration": 1 if source == "HISTORICAL" else 0,
        # Version Pointers (latest_major_version removed - derived from md_version_registry)
        "version": version,
        "current_draft_version": current_draft_version,
        "base_template_version": base_template_version,
        "notes": notes,
        # SCD Type 2 columns
        "effective_start_ts": now,  # When this DTA became active
        "effective_end_ts": None,   # NULL means currently active
        "is_current": True,         # Quick filter for active records
        # Audit columns
        "created_by_principal": created_by_principal,
        "created_ts": now,
        "last_updated_by_principal": created_by_principal,
        "last_updated_ts": now,
        "databricks_job_id": databricks_job_id,
        "databricks_job_name": databricks_job_name,
        "databricks_run_id": databricks_run_id
    }
    dta_records.append(dta_record)
    
    # For HISTORICAL, create auto-approved workflow record and approval task
    if source == "HISTORICAL":
        workflow_id = str(uuid.uuid4())
        
        # Create workflow record (auto-approved for historical)
        workflow_record = {
            "dta_workflow_id": workflow_id,
            "dta_id": dta_id,
            "workflow_iteration": 1,
            "workflow_status": "APPROVED",
            "summary_comment": "Historical DTA - auto-approved during import",
            "initiated_ts": now,
            "closed_ts": now,
            "created_by_principal": created_by_principal,
            "created_ts": now,
            "last_updated_by_principal": created_by_principal,
            "last_updated_ts": now,
            "databricks_job_id": databricks_job_id,
            "databricks_job_name": databricks_job_name,
            "databricks_run_id": databricks_run_id
        }
        workflow_records.append(workflow_record)
        
        # Create SYSTEM_APPROVED approval task (auto-approved for historical)
        # This represents the automatic DTA Major creation for pre-approved historical data
        # No JNJ_DAE/VENDOR tasks needed - those are only for UI manual approval flow
        approval_task_records.append({
            "approval_task_id": str(uuid.uuid4()),
            "dta_workflow_id": workflow_id,
            "dta_id": dta_id,
            "approver_role": "SYSTEM_APPROVED",
            "assigned_to_principal": None,  # System task, no user assigned
            "approval_status": "APPROVED",
            "approval_order": 99,  # Always last in sequence
            "is_auto_approve": True,
            "approval_comment": "Historical import - auto-promoted to DTA Major",
            "approved_ts": now
        })
    
    # Generate user-friendly DTA name
    # Historical: {trial_id}_{data_stream_type}_{data_provider_name}
    # UI: <Change_this>_{trial_id}_{data_stream_type}_{data_provider_name}
    if source == "HISTORICAL":
        dta_name = f"{trial_id}_{stream_type}_{provider}"
    else:
        dta_name = f"<Change_this>_{trial_id}_{stream_type}_{provider}"
    
    # Add dta_name to the record
    dta_record["dta_name"] = dta_name
    
    print(f"  Created DTA: {dta_number} (ID: {dta_id[:8]}...)")
    print(f"  Name: {dta_name}")
    print(f"  Status: {status}, Workflow State: {workflow_state}")


In [ ]:
# Cell 9: Save to Tables
print(f"\n{'='*80}")
print("Saving DTA records...")
print(f"{'='*80}")

if dta_records:
    # Save DTA records (with schema evolution for new columns like parent_document_id)
    dta_df = spark.createDataFrame(dta_records, schema=get_dta_schema())
    dta_df.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(dta_table)
    print(f"Saved {len(dta_records)} DTA record(s)")

if workflow_records:
    # Save workflow records
    wf_df = spark.createDataFrame(workflow_records, schema=get_dta_workflow_schema())
    wf_df.write.format("delta").mode("append").saveAsTable(dta_workflow_table)
    print(f"Saved {len(workflow_records)} workflow record(s)")

if approval_task_records:
    # Save approval task records using common audit framework
    # Audit columns (created_by_principal, created_ts, etc.) are added automatically
    task_df = spark.createDataFrame(approval_task_records, schema=get_dta_approval_task_schema())
    save_with_audit(
        df=task_df,
        table_name=dta_approval_task_table,
        created_by_principal=created_by_principal,
        databricks_job_id=databricks_job_id,
        databricks_job_name=databricks_job_name,
        databricks_run_id=databricks_run_id,
        mode="append"
    )
    print(f"Saved {len(approval_task_records)} approval task record(s) (SYSTEM_APPROVED auto-approved)")


In [ ]:
# Cell 10: Set Task Values for Downstream Tasks
# Store created DTA IDs and numbers for downstream processing
# NOTE: library_type is EMPTY to indicate ALL types should be processed
dbutils.jobs.taskValues.set(key="created_dta_ids", value=json.dumps(created_dta_ids))
dbutils.jobs.taskValues.set(key="created_dta_numbers", value=json.dumps(created_dta_numbers))
dbutils.jobs.taskValues.set(key="source", value=source)
dbutils.jobs.taskValues.set(key="library_type", value="")  # Empty = ALL library types
dbutils.jobs.taskValues.set(key="registry_table", value=registry_table)
dbutils.jobs.taskValues.set(key="dta_count", value=len(created_dta_ids))

print(f"\n{'='*80}")
print("Summary")
print(f"{'='*80}")
print(f"Source: {source}")
print(f"Library Types: ALL ({[lib.get('library_type') for lib in library_tables]})")
print(f"DTAs created: {len(created_dta_ids)}")
for dta_id, dta_num in zip(created_dta_ids, created_dta_numbers):
    print(f"  - {dta_num} (ID: {dta_id[:8]}...)")
